##Tesseract OCR para Imagens


In [1]:
!pip install opencv-python

Primeiro, vamos instalar as dependências necessárias:

In [2]:
!pip install pytesseract

In [3]:
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!apt-get install -y tesseract-ocr-por

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 3,743 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.5 [581 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB

Em seguida, podemos carregar a imagem e aplicar o OCR:

In [6]:
import pytesseract
import cv2
from google.colab.patches import cv2_imshow

# Carregando a imagem
img = cv2.imread('cnh.jpg')

# Check if the image was loaded successfully
if img is None:
    print("Error: Could not load the image. Please check the file path.")
else:
    # Convertendo para escala de cinza
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2_imshow(img_gray)

    # Aplicando o Tesseract OCR
    texto = pytesseract.image_to_string(img_gray, lang='por')
    print(texto)

Error: Could not load the image. Please check the file path.


In [ ]:
print(img)

In [ ]:
print(img_gray)

Pré-processamento e Melhoria de Imagem

In [ ]:
# Aplicando técnicas de thresholding
_, img_thresh = cv2.threshold(img_gray, 142, 255, cv2.THRESH_BINARY)
cv2_imshow(img_thresh)

# Extraindo texto após pré-processamento
texto_thresh = pytesseract.image_to_string(img_thresh, lang='por')
print(texto_thresh)

In [ ]:
# Aplicando o método de Otsu
_, img_otsu = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
cv2_imshow(img_otsu)

# Extraindo texto após pré-processamento
texto_otsu = pytesseract.image_to_string(img_otsu, lang='por')
print(texto_otsu)

In [ ]:
# Aplicando limiarização adaptativa
img_adaptive_mean = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 33, 9)
cv2_imshow(img_adaptive_mean)

# Extraindo texto após pré-processamento
texto_adaptive_mean = pytesseract.image_to_string(img_adaptive_mean, lang='por')
print(texto_adaptive_mean)


In [ ]:
# Aplicando filtros para remoção de ruído
img_median_blur = cv2.medianBlur(img_gray, 3)
cv2_imshow(img_median_blur)

# Extraindo texto após pré-processamento
texto_median_blur = pytesseract.image_to_string(img_median_blur, lang='por')
print(texto_median_blur)

## Analise da extração!

In [ ]:
from difflib import SequenceMatcher

# Texto original
texto_original = """
Você não pode colocar um limite
em nada. Quanto mais você sonha,
mais longe você chega.

Michael Phelps
"""

# Variáveis que contêm os textos OCR a serem verificados
texto_ocr_variants = {
    "texto": texto,
    "texto_thresh": texto_thresh,
    "texto_otsu": texto_otsu,
    "texto_adaptive_mean": texto_adaptive_mean,
    "texto_median_blur": texto_median_blur,
}

def compare_texts(original, ocr_text):
    matcher = SequenceMatcher(None, original, ocr_text)
    match_ratio = matcher.ratio()
    return match_ratio

def main():
    original_len = len(texto_original)
    print(f"Quantidade de caracteres no texto original: {original_len}\n")

    for variant_name, ocr_text in texto_ocr_variants.items():
        ocr_len = len(ocr_text)
        accuracy = compare_texts(texto_original, ocr_text)
        accuracy_percentage = accuracy * 100

        print(f"Resultado para {variant_name}:")
        print(f"Quantidade de caracteres no texto OCR: {ocr_len}")
        print(f"Percentual de acertos: {accuracy_percentage:.2f}%\n")


if __name__ == "__main__":
    main()


## Paddle OCR

In [ ]:
!pip install paddleocr==2.7.3
!pip install paddlepaddle==2.6.1

In [ ]:
from paddleocr import PaddleOCR

# Inicializa o objeto PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Função para extrair texto OCR de um PDF
def extract_text_from_pdf(pdf_path):
    # Use o método `ocr.ocr()` para extrair texto de uma imagem
    extract_ocr = ocr.ocr(pdf_path)  # Removido o argumento `lang`

    # Concatene todas as palavras detectadas em uma única string
    result = ''
    for line in extract_ocr:
        for word in line:
            result += word[1][0] + ' '  # O índice 0 representa o texto da palavra
        result += '\n'  # Adicione uma nova linha após cada linha de texto
    return result



In [ ]:
# from utils.AI_response_formatting import extract_fields
# from utils.ocr_extraction import OCRProcessor

# Caminho para o arquivo PDF
pdf_path = "/content/enel_ficticia.pdf"

# Extrai texto do PDF usando OCR
ocr_text = extract_text_from_pdf(pdf_path)

# Imprimir os campos extraídos
print(ocr_text)

In [ ]:
import re

# Expressões regulares para encontrar padrões específicos
padrao_cpf_cnpj = r"CPF/ CNPJ:\s([\d.-]+)"
padrao_nome_cliente = r"CPF:\s([\w\s]+)\s\d{3}\.\d{3}\.\d{3}-\d{2}"
padrao_num_cliente = r"NDO CLIENTE\s(\d+)"
padrao_data_leitura = r"DataLeitor\sLeitura\s(\d{2}/\d{2}/\d{4})"
padrao_data_vencimento = r"MES/AN\sVENCIMENT\sTOTAL\sA\sPAGAF\s(\d{2}/\d{4}\s\d{2}/\d{2}/\d{4})"
padrao_valor_total = r"R\$(\d+,\d{2})"
padrao_descricao_faturamento = r"DESCRICAO DO FATURAMENTO(.*?)MENTOS DE MEI"
padrao_referencia_faturamento = r"Referencia\s(\d{2}/\d{4}\s\d{2}/\d{2}/\d{4})"
padrao_qr_code = r"Pague via PIX! Utilize este QR Code"

# Função para extrair informações com base nos padrões de expressão regular
def extrair_informacoes(texto, padrao):
    match = re.search(padrao, texto)
    if match:
        return match.group(1)
    else:
        return None

# Extrair informações
cpf_cnpj = extrair_informacoes(ocr_text, padrao_cpf_cnpj)
nome_cliente = extrair_informacoes(ocr_text, padrao_nome_cliente)
num_cliente = extrair_informacoes(ocr_text, padrao_num_cliente)
data_vencimento = extrair_informacoes(ocr_text, padrao_data_vencimento)
valor_total = extrair_informacoes(ocr_text, padrao_valor_total)
referencia_faturamento = extrair_informacoes(ocr_text, padrao_referencia_faturamento)

# Verificar se há QR Code presente
qr_code_presente = re.search(padrao_qr_code, ocr_text) is not None

# Exibir informações extraídas
print("Nome do Cliente:", nome_cliente)
print("CPF/CNPJ:", cpf_cnpj)
print("Número do cliente:", num_cliente)
print("Data de vencimento:", data_vencimento)
print("Valor total:", valor_total)
print("Referência do faturamento:", referencia_faturamento)
print("QR Code presente:", qr_code_presente)


In [ ]:
# Make sure to load the image for PaddleOCR as well, if needed
# For example, if you're processing the same image 'cnh.png':
# img_paddle = cv2.imread('cnh.png')

# If you are processing a PDF file, the PaddleOCR library handles the image loading internally.
# Make sure the path to your PDF file is correct.
# pdf_path = "/content/enel_ficticia.pdf"

# If you were trying to use the 'img' variable from the previous section with PaddleOCR,
# you would need to ensure 'img' was loaded successfully there.